# Send REST requests to the WeatherApp.API running in Aspire

In [1]:
// First set the base url.
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [2]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

Name,Value
traceparent,00-65ff3966176893049de73c95a31953b0-10a1f5b85493c1b2-00
Name,Value
Date,"Sat, 19 Apr 2025 21:46:14 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,text/plain; charset=utf-8
Content-Length,26


## GET Weather Forecast 🌦️🌈🔆

In [3]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

Name,Value
traceparent,00-d9990f76376edc71db4a5d26384ad1af-73929a0db81fbc5e-00
Name,Value
Date,"Sat, 19 Apr 2025 21:46:21 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,175
requestedRegion,"""Bristol"""
requestedDate,"""2025-04-19T21:46:21.1854323Z"""
requestId,"""225cc391-85c7-43a5-b1e6-612d7faf2c42"""


## POST Colected Weather Data 🌡️☔💨

In [4]:
// Generate a random reference...
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

watermelon-91

In [7]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
x-request-id: {{$guid}}
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Name,Value
X-Request-ID,a31f9e6a-ed2c-4aba-9e0d-4c9b19e36b0f
traceparent,00-5de750c916d826d6e5703b89af3a8ee7-3f0da0f6b1d0985c-00
Content-Type,application/json
Content-Length,290
Name,Value
Date,"Sat, 19 Apr 2025 21:50:07 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,80


<div style="display:block; height:40rem;"></div>

## GET Traces 🤖

In [9]:
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notication Sent")
            thirdPart = $"✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
        
            
        
        return $"{Resource.PadRight(30, '-')}> {DisplayName.PadRight(30, '-')}> {thirdPart}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("https://localhost:7059") };
var response = await client.GetAsync("traces");
var responseJson = await response.Content.ReadAsStringAsync();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();

index value 0 NotificationService-----------> User Notication Sent----------> ✉️ reference: watermelon-91 Resource NotificationService Source UserNotificationEventHandler DisplayName User Notication Sent TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId e5b9b7153c4221f7 Tags key value user-notification-event.body "Dear user, your data has been submitted and included in our latest model" user-notification-event.reference "watermelon-91" user-notification-event.timestamp "2025-04-19T21:50:10.0622609+00:00" 1 WeatherApp.EventListener------> Outbox Item Insertion---------> 📫 outbox id: 20002 Resource WeatherApp.EventListener Source OutboxRepository DisplayName Outbox Item Insertion TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId 22a185c7ea382957 Tags key value outbox-item.TypeName "UserNotificationEvent" outbox-item.Id 20002 2 WeatherApp.EventListener------> Domain Event Insertion--------> ModelUpdated Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId 0cfdd9d2d68e89ef Tags key value domain-event.streamId "bc1127cd-e003-4c3e-871b-ca284423c554" domain-event.version "8" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelUpdated" 3 WeatherApp.EventListener------> Domain Event Insertion--------> PendingContributorPaymentCommitted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId a636a44d345572b5 Tags key value domain-event.streamId "bc1127cd-e003-4c3e-871b-ca284423c554" domain-event.version "6" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentCommitted" 4 WeatherApp.API----------------> Domain Event Insertion--------> PendingContributorPaymentPosted Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId 67ee2ec463f04395 Tags key value domain-event.streamId "bc1127cd-e003-4c3e-871b-ca284423c554" domain-event.version "3" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentPosted" 5 WeatherApp.EventListener------> Domain Event Insertion--------> SubmissionComplete Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId 1213d208b8355e63 Tags key value domain-event.streamId "bc1127cd-e003-4c3e-871b-ca284423c554" domain-event.version "7" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmissionComplete" 6 WeatherApp.EventListener------> Domain Event Insertion--------> ModelingDataAccepted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId 324451837617748f Tags key value domain-event.streamId "bc1127cd-e003-4c3e-871b-ca284423c554" domain-event.version "5" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelingDataAccepted" 7 WeatherApp.API----------------> Domain Event Insertion--------> SubmittedToModeling Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId ed98c5f9392e2d8d Tags key value domain-event.streamId "bc1127cd-e003-4c3e-871b-ca284423c554" domain-event.version "4" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmittedToModeling" 8 WeatherApp.API----------------> Domain Event Insertion--------> LocationIdFound Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 5de750c916d826d6e5703b89af3a8ee7 SpanId f614bb97fcfdaf21 Tags key value domain-event.streamId "bc1127cd-e003-4c3e-871b-ca284423c554" domain-event.version "2" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.LocationIdFound" 9 WeatherApp.API----------------> Domain Event Insertion--------> WeatherDataCollectionInitiated Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event